# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,274.60,64,96,10.18,TF,1727664479
1,1,tazovsky,67.4667,78.7000,280.01,92,100,5.86,RU,1727664480
2,2,sitka,57.0531,-135.3300,285.09,71,75,0.00,US,1727664483
3,3,igarka,67.4667,86.5833,280.65,95,100,2.08,RU,1727664484
4,4,makurazaki,31.2667,130.3167,302.83,56,1,3.54,JP,1727664485


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Applying the Kelvin to Celsius conversion
city_data_df['Max Temp'] = city_data_df['Max Temp'] - 273.15

# Ensuring all Max Temp values are numeric
city_data_df['Max Temp'] = pd.to_numeric(city_data_df['Max Temp'], errors='coerce')

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,1.45,64,96,10.18,TF,1727664479
1,1,tazovsky,67.4667,78.7000,6.86,92,100,5.86,RU,1727664480
2,2,sitka,57.0531,-135.3300,11.94,71,75,0.00,US,1727664483
3,3,igarka,67.4667,86.5833,7.50,95,100,2.08,RU,1727664484
4,4,makurazaki,31.2667,130.3167,29.68,56,1,3.54,JP,1727664485


In [7]:
print(city_data_df['Max Temp'].describe())

count    530.000000
mean      17.876604
std        9.370568
min       -6.710000
25%       10.307500
50%       19.320000
75%       25.640000
max       38.990000
Name: Max Temp, dtype: float64


In [8]:
# Now apply the filtering criteria
ideal_cities_df = city_data_df[(city_data_df['Max Temp'] > 21.00) & (city_data_df['Max Temp'] < 27.00) &
                               (city_data_df['Wind Speed'] < 4.5) &
                               (city_data_df['Cloudiness'] <= 0.01)]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,tolanaro,-25.0319,46.9987,22.04,88,0,2.06,MG,1727664517
31,31,crane,31.3974,-102.3501,24.48,28,0,4.08,US,1727664520
47,47,tanumah,27.1000,44.1333,24.27,19,0,3.57,SA,1727664543
66,66,san patricio,28.0170,-97.5169,23.97,59,0,3.32,US,1727664567
68,68,fort stockton,30.8940,-102.8793,26.03,24,0,1.54,US,1727664569
105,105,pace,30.5994,-87.1611,25.77,78,0,3.60,US,1727664617
106,106,at taj,24.2049,23.2857,22.83,37,0,3.73,LY,1727664619
142,142,geraldton,-28.7667,114.6000,24.23,35,0,3.60,AU,1727664667
165,165,kas,36.2018,29.6377,23.00,59,0,2.20,TR,1727664635
172,172,newman,37.3138,-121.0208,23.91,56,0,0.89,US,1727664709


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
29,tolanaro,MG,-25.0319,46.9987,88,
31,crane,US,31.3974,-102.3501,28,
47,tanumah,SA,27.1000,44.1333,19,
66,san patricio,US,28.0170,-97.5169,59,
68,fort stockton,US,30.8940,-102.8793,24,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:

# Set the parameters for the type of place
categories = "accommodation.hotel"
radius = 10000

params = {
    "categories":categories,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    data = response.json()

    # Check if any hotels were found and if the first result has a "name" property
    if "features" in data and len(data["features"]) > 0 and "name" in data["features"][0]["properties"]:
        # Get the name of the first hotel in the results
        hotel_name = data["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    else:
        # If no hotel is found or the "name" property is missing, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tolanaro - nearest hotel: Hôtel Mahavokey
crane - nearest hotel: No hotel found
tanumah - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
fort stockton - nearest hotel: La Quinta Inn
pace - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Hotel
kas - nearest hotel: KEKOVA OTEL
newman - nearest hotel: No hotel found
pratt - nearest hotel: Pratt Inn & Suites
la porte - nearest hotel: Garden Suites Hotel
maintirano - nearest hotel: Hotel Safari
blair - nearest hotel: No hotel found
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
tripoli - nearest hotel: الاصيل
childress - nearest hotel: Red Roof Inn Childress
heroica matamoros - nearest hotel: Cameron Hotel
mims - nearest hotel: Executive Garden Titusville Hotel
aswan - nearest hotel: Oscar Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
29,tolanaro,MG,-25.0319,46.9987,88,Hôtel Mahavokey
31,crane,US,31.3974,-102.3501,28,No hotel found
47,tanumah,SA,27.1000,44.1333,19,No hotel found
66,san patricio,US,28.0170,-97.5169,59,No hotel found
68,fort stockton,US,30.8940,-102.8793,24,La Quinta Inn
105,pace,US,30.5994,-87.1611,78,No hotel found
106,at taj,LY,24.2049,23.2857,37,No hotel found
142,geraldton,AU,-28.7667,114.6000,35,Ocean Centre Hotel
165,kas,TR,36.2018,29.6377,59,KEKOVA OTEL
172,newman,US,37.3138,-121.0208,56,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',  # Longitude and Latitude for map plot
    geo=True,      # Enables a geographical plot
    tiles='OSM',   # OpenStreetMap tiles for the background
    size=100,      # Size of points on the map
    color='blue',  # Color of points
    hover_cols=['City', 'Country', 'Hotel Name'],  # Hover info: City, Country, and Hotel Name
    frame_width=800,  # Width of the plot
    frame_height=600  # Height of the plot
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)